## TITLE: Resave Gauge Data

**Purpose:** 

**Dependencies:** 

---


In [1]:
import numpy as np
import pandas as pd
import glob
import os
import tarfile

In [2]:
def load_gauge_data(allfiles, no_breach):
    df_list = []
    for file in allfiles:
        sim_name = file.split('/')[-3]
        cols = ['Time', f'{sim_name}']
        df = pd.read_csv(file, skiprows=3, header=None,
                         delim_whitespace=True, usecols=[1,5],
                         names = cols, index_col='Time')
        if sim_name == 'no_breach':
            no_breach = df
        df_list.append(df)
    df_list.append(no_breach)
        
    df_list = reindex_df(df_list, no_breach)
    data_df = merge_clean_data(df_list, 486)
    return data_df

def merge_clean_data(data_dict, storm_id):
    """
    Takes a dictionary of dataframes, concatenates them together and
    Changes time from seconds to hours from landfall
    """
    df = pd.concat(data_dict, axis=1)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(0)
    landfall_time = get_landfall_time(storm_id)
    df.index -= landfall_time
    df.index /= 3600.0
    return df


def get_landfall_time(storm_id):
    import xarray as xr
    storm_data = xr.open_dataset(os.path.join('/projects/weiszr_lab/catherine/storm_files', 
                                              f'NACCS_TP_{storm_id:04}_SYN_L2.nc'))
    landfall_loc =  [-73.31, 40.68]
    eye = storm_data.eye_loc.data
    lat_row = np.where(eye[:,1] >= 40.68)
    lon_row = np.where(eye[:,0] <= -73.31)
    idx = max(min(lat_row[0]), min(lon_row[0]))
    landfall_time = storm_data.time[idx].values
    return landfall_time

    
def reindex_df(df_list, res_15):
    df_l = []
    for df in df_list:
        df_re = df.reindex(df.index.union(res_15.index)).interpolate('index').reindex(res_15.index)
        df_l.append(df_re)
    return df_l

In [4]:
df = pd.read_pickle('/projects/weiszr_lab/catherine/breach_sims/15min_output/no_breach/_output/gauge50084.pkl.gz')

In [3]:
PATH = '/projects/weiszr_lab/catherine/breach_sims/'
gauges_to_save = {'west' : [84, 82],
                  'central' : [133, 45],
                  'east' : [11, 119]}
gauge_data = {}
for location in gauges_to_save:
    for gauge in gauges_to_save[location]:

        gaugefile = f'gauge1{gauge:04}.txt'
        no_breach = pd.read_pickle(f'/home/catherinej/no_breach/_output/gauge1{gauge:04}.pkl.gz')
        no_breach = no_breach.rename(columns={'Eta': 'no_breach'})
        files = glob.glob(os.path.join(PATH, '**', gaugefile), recursive=True)
        gaugefiles = [file for file in files if 'r' not in file.split('/')[-1]]
        
        data = load_gauge_data(gaugefiles, no_breach)
        if 'no_breach' in data.columns:
            print('exists')
        data.to_pickle(f'/home/catherinej/{location}_gauge1{gauge:04}.pkl.gz', compression='gzip')

exists
exists
exists
exists
exists
exists


In [4]:
PATH = '/projects/weiszr_lab/catherine/breach_sims/'
gauges_to_save = {'west' : [84, 82, 80],
                  'central' : [79, 133, 45],
                  'east' : [11, 105, 119]}
gauge_data = {}
g_data = np.loadtxt('/projects/weiszr_lab/catherine/multi/m_gauges.csv',
                    skiprows=1, usecols=(1,2), delimiter=',')
for location in gauges_to_save:
    print(location)
    loc = {}
    for gauge in gauges_to_save[location]:
        loc[gauge] = g_data[gauge]
        gauge_data[location] =loc

west
central
east


In [5]:
df = pd.DataFrame.from_dict({(i,j): gauge_data[i][j] 
                           for i in gauge_data.keys() 
                           for j in gauge_data[i].keys()},
                       orient='index')
df = df.rename(columns={0:'lon', 1:'lat'})

In [6]:
df

,lon,lat
"(west, 84)",-72.85546,40.740220
"(west, 82)",-72.83375,40.753260
"(west, 80)",-72.80720,40.764400
"(central, 79)",-72.81200,40.778100
"(central, 133)",-72.75266,40.781300
"(central, 45)",-72.75000,40.786667
"(east, 11)",-72.70000,40.795000
"(east, 105)",-72.67050,40.798000
"(east, 119)",-72.65900,40.797600


In [32]:
df.to_csv('plotting_gauges_locs.csv')

In [4]:
cols = ['Time', 'Eta']
# os.mkdir('compressed_g')
for d in dirs:
    if not d.split('/')[-1].isalpha():
        print(f'Working on folder: {d}')
        all_gauges = glob.glob(os.path.join(d, '_output', 'gauge*.txt'), recursive=True)
        gauges = load_gauge_data(all_gauges)
#         if all_gauges:
#             for gauge in all_gauges:
#                 name = gauge.split('/')[-1].split('.')[0]
                
#                 df = pd.read_csv(gauge, skiprows=3, header=None, 
#                                  delim_whitespace=True, usecols=[1,5], 
#                                  index_col='Time', names=cols)
#                 df.to_pickle(os.path.join(d, '_output', name + '.pkl.gz'), compression='gzip')
#                 os.remove(gauge)
#         output_filename = os.path.join('/projects/weiszr_lab/catherine/breach_sims/sims',
#                                        d.split('/')[-1] + '.tar.gz')
#         if not os.path.exists(output_filename):
#             print(f'Making tar file: {output_filename}')
#             make_tarfile(output_filename, d)

Working on folder: /projects/weiszr_lab/catherine/breach_sims/sims/r158
Working on folder: /projects/weiszr_lab/catherine/breach_sims/sims/wr12
Working on folder: /projects/weiszr_lab/catherine/breach_sims/sims/d15
Working on folder: /projects/weiszr_lab/catherine/breach_sims/sims/dr11
Working on folder: /projects/weiszr_lab/catherine/breach_sims/sims/r184
Working on folder: /projects/weiszr_lab/catherine/breach_sims/sims/r144
Working on folder: /projects/weiszr_lab/catherine/breach_sims/sims/r199
Working on folder: /projects/weiszr_lab/catherine/breach_sims/sims/r159
Working on folder: /projects/weiszr_lab/catherine/breach_sims/sims/r119
Working on folder: /projects/weiszr_lab/catherine/breach_sims/sims/r170
Working on folder: /projects/weiszr_lab/catherine/breach_sims/sims/r130
Working on folder: /projects/weiszr_lab/catherine/breach_sims/sims/r48
Working on folder: /projects/weiszr_lab/catherine/breach_sims/sims/r145
Working on folder: /projects/weiszr_lab/catherine/breach_sims/sims

In [5]:
all_gauges = glob.glob(os.path.join(d, '_output', 'gauge*.txt'), recursive=True)